# Create Custom Sentinel-2 (COG) Tiler

Note: See https://github.com/developmentseed/titiler-pds for a end-to-end implementation

### Sentinel 2

Thanks to Digital Earth Africa and in collaboration with Sinergise, Element 84, Amazon Web Services (AWS) and the Committee on Earth Observation Satellites (CEOS), Sentinel 2 (Level 2) data over Africa, usually stored as JPEG2000, has been translated to COG. More importantly, a STAC database and API has been set up. 

https://www.digitalearthafrica.org/news/operational-and-ready-use-satellite-data-now-available-across-africa

The API is provided by [@element84](https://www.element84.com) and follows the latest specification: https://earth-search.aws.element84.com/v0


## Requirements

To be able to run this notebook you'll need the following requirements:
- rasterio
- folium
- requests
- titiler
- uvicorn
- rio-tiler-pds
- cogeo-mosaic

`!pip install titiler.core titiler.mosaic uvicorn folium requests rio-tiler-pds`

In [ ]:
# Uncomment this line if you need to install the dependencies
# !pip install titiler.core titiler.mosaic uvicorn folium requests rio-tiler-pds

## 1. Create the CustomTiler

In order to create our custom tiler we will use the rio-tiler-pds `S2COGReader` and the different Tiler Factories from titiler.

```python
"""
Sentinel 2 (COG) Tiler.

Based on rio-tiler-pds and titiler

>>> pip install titiler rio-tiler-pds

"""
from titiler.core.factory import MultiBandTilerFactory
from titiler.core.dependencies import BandsExprParams
from titiler.mosaic.factory import MosaicTilerFactory

from rio_tiler_pds.sentinel.aws import S2COGReader
from rio_tiler_pds.sentinel.utils import s2_sceneid_parser

from fastapi import FastAPI, Query


def CustomPathParams(
    sceneid: str = Query(..., description="Sentinel 2 Sceneid.")
):
    """Create dataset path from args"""
    assert s2_sceneid_parser(sceneid)  # Makes sure the sceneid is valid
    return sceneid


app = FastAPI()

scene_tiler = MultiBandTilerFactory(reader=S2COGReader, path_dependency=CustomPathParams, router_prefix="scenes")
app.include_router(scene_tiler.router, prefix="/scenes", tags=["scenes"])

mosaic_tiler = MosaicTilerFactory(
    router_prefix="mosaic",
    dataset_reader=S2COGReader,
    layer_dependency=BandsExprParams,
)
app.include_router(mosaic_tiler.router, prefix="/mosaic", tags=["mosaic"])
```

## 2. Launch the tiler

Save the python code from 1. into a file named app.py and then start the server within another terminal

```
$ uvicorn app:app
```

## 3. Search Data

In [ ]:
import os
import json
import base64
import requests
import datetime
import itertools
import urllib.parse
import pathlib

from io import BytesIO
from functools import partial
from concurrent import futures

from rasterio.plot import reshape_as_image
from rasterio.features import bounds as featureBounds

from folium import Map, TileLayer, GeoJson

%pylab inline

In [ ]:
# Endpoint variables
titiler_endpoint = "http://127.0.0.1:8000"
stac_endpoint = "https://earth-search.aws.element84.com/v0/search"

# Make sure both are up
assert requests.get(f"{titiler_endpoint}/docs").status_code == 200
assert requests.get(stac_endpoint).status_code == 200

More info: https://github.com/radiantearth/stac-api-spec for more documentation about the stac API

1. AOI

You can use geojson.io to define your search AOI

In [ ]:
geojson = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -2.83447265625,
              4.12728532324537
            ],
            [
              2.120361328125,
              4.12728532324537
            ],
            [
              2.120361328125,
              8.254982704877875
            ],
            [
              -2.83447265625,
              8.254982704877875
            ],
            [
              -2.83447265625,
              4.12728532324537
            ]
          ]
        ]
      }
    }
  ]
}

bounds = featureBounds(geojson)

m = Map(
    tiles="OpenStreetMap",
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=6
)

geo_json = GeoJson(data=geojson)
geo_json.add_to(m)
m

2. Define dates and other filters

In [ ]:
start = datetime.datetime.strptime("2019-01-01", "%Y-%m-%d").strftime("%Y-%m-%dT00:00:00Z")
end = datetime.datetime.strptime("2019-12-11", "%Y-%m-%d").strftime("%Y-%m-%dT23:59:59Z")

# POST body
query = {
    "collections": ["sentinel-s2-l2a-cogs"],
    "datetime": f"{start}/{end}",
    "query": {
        "eo:cloud_cover": {
            "lt": 3
        },
        "sentinel:data_coverage": {
            "gt": 10
        }
    },
    "intersects": geojson["features"][0]["geometry"],
    "limit": 1000,
    "fields": {
      'include': ['id', 'properties.datetime', 'properties.eo:cloud_cover'],  # This will limit the size of returned body
      'exclude': ['assets', 'links']  # This will limit the size of returned body
    },
    "sortby": [
        {
            "field": "properties.eo:cloud_cover",
            "direction": "desc"
        },
    ]
}

# POST Headers
headers = {
    "Content-Type": "application/json",
    "Accept-Encoding": "gzip",
    "Accept": "application/geo+json",
}

data = requests.post(stac_endpoint, headers=headers, json=query).json()
print("Results context:")
print(data["context"])

sceneid = [f["id"] for f in data["features"]]
cloudcover = [f["properties"]["eo:cloud_cover"] for f in data["features"]]
dates = [f["properties"]["datetime"][0:10] for f in data["features"]]

In [ ]:
data["features"][0]

In [ ]:
m = Map(
    tiles="OpenStreetMap",
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=8
)

geo_json = GeoJson(
    data=data,
    style_function=lambda x: {
        'opacity': 1, 'dashArray': '1', 'fillOpacity': 0, 'weight': 1
    },
)
geo_json.add_to(m)
m

## 4. Visualize One Item

In [ ]:
# Print what bands are available
from rio_tiler_pds.sentinel.aws.sentinel2 import default_l2a_bands
print(default_l2a_bands)

In [ ]:
# Fetch TileJSON
# For this example we use the first `sceneid` returned from the STAC API
# and we sent the Bands to B04,B03,B02 which are red,green,blue
data = requests.get(f"{titiler_endpoint}/scenes/tilejson.json?sceneid={sceneid[4]}&bands=B04,B03,B02&rescale=0,2000").json()
print(data)

In [ ]:
bounds = data["bounds"]
m = Map(
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=10
)

tiles = TileLayer(
    tiles=data["tiles"][0],
    min_zoom=data["minzoom"],
    max_zoom=data["maxzoom"],
    opacity=1,
    attr="Digital Earth Africa"
)
tiles.add_to(m)
m

## 5. Mosaic

Working with 1 scene is great, but working with 10s is better. Lets see how to create mosaics and use it with our new tiler.

### 5.1 Create a MosaicJSON document


To create a mosaicJSON, we can use developmentseed/cogeo-mosaic python module. Within this module there is a `backend` which is able to work with the STAC API directly to create mosaicJSON.

In [ ]:
from cogeo_mosaic.backends import MosaicBackend


def custom_accessor(feature):
    """Return feature identifier."""
    return feature["id"]

with MosaicBackend(
    "stac+https://earth-search.aws.element84.com/v0/search",
    query,
    minzoom=8,
    maxzoom=15,
    backend_options={"accessor": custom_accessor},
) as mosaic:
    print(mosaic.metadata)
    mosaic_doc = mosaic.mosaic_def.dict(exclude_none=True)

In [ ]:
mosaic_doc

### 5.2 Custom Filter



As we can see in the mosaicJSON, each `quadkey` index has 10s scenes. Because we want to build a visual mosaic, we don't need as many assets and maybe 1 or 2 scenes per quadkey will be enought to create mercator tiles. To reduce the number of scenes per quadkey we need to apply a custom filter while building the mosaic.

In [ ]:
from typing import Dict, List, Sequence, Optional
from pygeos import polygons
import mercantile


def optimized_filter(
    tile: mercantile.Tile,  # noqa
    dataset: Sequence[Dict],
    geoms: Sequence[polygons],
    minimum_tile_cover=None,  # noqa
    tile_cover_sort=False,  # noqa
    maximum_items_per_tile: Optional[int] = None,
) -> List:    
    """Optimized filter that keeps only one item per grid ID."""
    gridid: List[str] = []
    selected_dataset: List[Dict] = []
    
    for item in dataset:
        grid = item["id"].split("_")[1]
        if grid not in gridid:
            gridid.append(grid)
            selected_dataset.append(item)

    dataset = selected_dataset

    indices = list(range(len(dataset)))
    if maximum_items_per_tile:
        indices = indices[:maximum_items_per_tile]

    return [dataset[ind] for ind in indices]



with MosaicBackend(
    "stac+https://earth-search.aws.element84.com/v0/search",
    query,
    minzoom=8,
    maxzoom=14,
    backend_options={"accessor": custom_accessor, "asset_filter": optimized_filter},
) as mosaic:
    print(mosaic.metadata)
    mosaic_doc = mosaic.mosaic_def.dict(exclude_none=True)

In [ ]:
mosaic_doc

### 5.3 Save Mosaic

In [ ]:
mosaic_file = "mymosaic.json.gz"
with MosaicBackend(mosaic_file, mosaic_def=mosaic_doc) as mosaic:
    mosaic.write(overwrite=True)

In [ ]:
# We need to pass the absolute path to the tiler
mosaic = str(pathlib.Path(mosaic_file).absolute())

### 5.4 Use It

In [ ]:
data = requests.get(f"{titiler_endpoint}/mosaic/info?url=file:///{mosaic}").json()
print(data)

In [ ]:
data = requests.get(f"{titiler_endpoint}/mosaic/tilejson.json?url=file:///{mosaic}&bands=B01&rescale=0,1000").json()

bounds = data["bounds"]
m = Map(
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=10
)

tiles = TileLayer(
    tiles=data["tiles"][0],
    min_zoom=data["minzoom"],
    max_zoom=data["maxzoom"],
    opacity=1,
    attr="Digital Earth Africa"
)
tiles.add_to(m)
m